## A module for creating and training a neural network for handwritten digit recognition using the gradient descent method.
95 % accuracy after 30 epochs

In [ ]:
import random
import numpy as np


""" ---Description--- """
def sigmoid(z): # definition of sigmoidal activation function
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):# Derivative of the sigmoidal function
    return sigmoid(z)*(1-sigmoid(z))


""" --Decription of the class Network--"""
class Network(object): # used to describe a neural network
    def __init__(self, sizes):  # sizes – list of sizes of neural network layers
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] # set random initial biases
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])] # set random initial weights

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(  # Stochastic gradient descent
            self,
            training_data,
            epochs,
            mini_batch_size,    # subsample size
            eta,                # learning speed rate
            test_data):

        test_data = list(test_data)
        n_test = len(test_data)
        training_data = list(training_data)
        n = len(training_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k + mini_batch_size] for k in\
                            range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)  # one step of a gradient descent
            print("Epoch {0}: accuracy is {1}/{2}".format(j, self.evaluate(test_data), n_test))  # print learning progress

    def update_mini_batch(  # gradient descent step
            self,
            mini_batch,
            eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]  # list of dC / db gradients for each layer
                                                            # (initially filled with zeros)
        nabla_w = [np.zeros(w.shape) for w in self.weights]  # list of dC / dw gradients for each layer
                                                             # (initially filled with zeros)
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)  # calculate the gradients dC / db and dC / dw
                                                                # layer by layer for the current use case (x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]  # summarizing the dC / db gradients
                                                                             # for different use cases of the current subsample
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]  # summing the dC / dw gradients
                                                                             # for various use cases of the current subsample
        self.weights = [w - (eta / len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)]  # update all weights
        self.biases = [b - (eta / len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)]  # update all biases
        
    def backprop(  # Backpropagation algorithm
            self,
            x,  # input signal vector
            y):  # expected vector of outputs
        nabla_b = [np.zeros(b.shape) for b in self.biases]  # list of dC / db gradients for each layer
                                                            # (initially filled with zeros)
        nabla_w = [np.zeros(w.shape) for w in self.weights]  # list of dC / dw gradients for each layer
                                                             # (initially filled with zeros)

        activation = x  # layer outputs (initially corresponds to layer 1 outputs or network inputs)
        activations = [x]  # list of output signals for all layers (initially contains only output signals of the 1st layer)
        zs = []  # list of activation potentials for all layers (initially empty)

        # direct distribution
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b  # count the activation potentials of the current layer
            zs.append(z)  # add an element (layer activation potentials) to the end of the list
            activation = sigmoid(z)  # count the output signals of the current layer by applying
                                     # the sigmoidal activation function to the activation potentials of the layer
            activations.append(activation)  # add an element (layer outputs) to the end of the list

        # backpropagation
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])  # consider the measure of the influence
                                                                    # of neurons in the output layer L on the error value (BP1)
        nabla_b[-1] = delta  # dC / db gradient for layer L (BP3)
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())  # dC / dw gradient for layer L (BP4)
        for l in range(2, self.num_layers):
            z = zs[-l]  # activation potentials of the l-layer (move through the list from right to left)
            sp = sigmoid_prime(z)  # consider the sigmoidal function of the activation potentials of the l-layer
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp  # we consider the measure of the influence
                                                                    # of the neurons of the l-th layer on the error value (BP2)
            nabla_b[-l] = delta  # dC / db gradient for layer l (BP3)
            nabla_w[-l] = np.dot(delta, activations[-l - 1].transpose())  # dC / dw gradient for layer l (BP4)
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):  # Assessment of learning progress
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):  # Calculation of partial derivatives of the cost function
                                                       # from the output signals of the last layer
        return (output_activations - y)


""" ---Neural Network--- """
net = Network([2, 3, 1]) # creating a neural network of three layers


""" Printing Neural Network info: """
print('Neural Net:')
print('Number of layers:', net.num_layers)
for i in range(net.num_layers):
    print('Number of neurons in a layer', i, ':', net.sizes[i])
for i in range(net.num_layers-1):
    print('W_', i+1, ':')
    print(np.round(net.weights[i], 2))
    print('b_', i+1, ':')
    print(np.round(net.biases[i], 2))

In [ ]:
"""
Connecting and using the MNIST database.
"""
import gzip # library for compressing and decompressing gzip and gunzip files
import pickle # library for saving and loading complex Python objects


def load_data():
    f = gzip.open('mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
    f.close()
    return training_data, validation_data, test_data


def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    
    # create sets of training, validation and test data
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return training_data, validation_data, test_data


def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


In [ ]:
# from Network1 import mnist_loader, network

training_data, validation_data, test_data = load_data_wrapper()
net = Network([784, 30, 10])
"""
The parameters specified when calling this method determine the topology of the created network.
Thus, as a result of the command execution, a network will be created, consisting of three layers:
the input layer of the network consists of 784 neurons; an inner layer of 30 neurons and an output layer of 10 neurons.
"""
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)